In [ ]:
!pip install -qU diffusers transformers huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Reproducible pipelines

While we cannot expect to get identical results using diffusion models, we can expect reproducible results across releases and platforms within a certain tolerance range.

## Control randomness

During inference, pipelines rely heavily on random sampling operations which include creating the Gaussian noise tensors to denoise and adding noise to the scheduling step.

In [ ]:
from diffusers import DDIMPipeline
import numpy as np

ddim = DDIMPipeline.from_pretrained(
    'google/ddpm-cifar10-32',
    use_safetensors=True
)

In [ ]:
image = ddim(
    num_inference_steps=2,
    output_type='np'
).images

# we get a different value everytime we run it
print(np.abs(image).sum())

Each time the pipeline is run, `torch.randn` uses a different random seed to create the Gaussian noise tensors.

##### CPU

In [ ]:
import torch
import numpy as np
from diffusers import DDIMPipeline

ddim = DDIMPipeline.from_pretrained(
    'google/ddpm-cifar10-32',
    use_safetensors=True
)
generator = torch.Generator('cpu').manual_seed(111)

In [ ]:
image = ddim(
    num_inference_steps=2,
    output_type='np',
    generator=generator
).images
print(np.abs(image).sum())

##### GPU

Full reproducibility across different hardware is not guaranteed because matrix multiplication.

If we run the same code example from the CPU example, we will get a different result even though the seed is identical.

In [ ]:
import torch
import numpy as np
from diffusers import DDIMPipeline

ddim = DDIMPipeline.from_pretrained(
    'google/ddpm-cifar10-32',
    use_safetensors=True
).to('cuda')
generator = torch.Generator('cuda').manual_seed(111)

In [ ]:
image = ddim(
    num_inference_steps=2,
    output_type='np',
    generator=generator
).images
print(np.abs(image).sum())

To avoid this issue, Diffusers has a `randn_tensor()` function for creating random noise on the CPU, and then moving the tensor to a GPU if necessary. The `randn_tensor()` function is used everywhere inside the pipeline.

In [ ]:
import torch
import numpy as np
from diffusers import DDIMPipeline

ddim = DDIMPipeline.from_pretrained(
    'google/ddpm-cifar10-32',
    use_safetensors=True
).to('cuda')

In [ ]:
generator = torch.manual_seed(111)

image = ddim(
    num_inference_steps=2,
    output_type='np',
    generator=generator,
).images
print(np.abs(image).sum())

## Deterministic algorithms

The downside to create a reproducible pipeline is that deterministic algorithms may be slower than non-deterministic ones and we may observe a decrease in performance.

PyTorch typically benchmarks multiple algorithms to select the fastest one, but if we want reproducibility, we should disable this feature. Set Diffusers:
```python
enable_full_determinism()
```


In [1]:
from diffusers.utils.testing_utils import enable_full_determinism
enable_full_determinism()

Now when we run the same pipeline twice, we will get identical results.

In [ ]:
import torch
from diffusers import DDIMScheduler, StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained(
    'stable-diffusion-v1-5/stable-diffusion-v1-5',
    use_safetensors=True,
).to('cuda')
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)

g = torch.Generator('cuda')
prompt = "A bear is playing a guitar on Times Square"

In [ ]:
g.manual_seed(111)
result1 = pipeline(
    prompt,
    num_inference_steps=50,
    generator=g,
    output_type='latent'
).images

In [ ]:
g.manual_seed(111)
result2 = pipeline(
    prompt,
    num_inference_steps=50,
    generator=g,
    output_type='latent'
).images

In [ ]:
print("L_inf dist =", abs(result1 - result2).max())

## Deterministic batch generation

A practical application of creating reproducible pipelines is *deterministic batch generation*.

In [ ]:
import torch
from diffusers import DiffusionPipeline
from diffusers.utils import make_image_grid

pipeline = DiffusionPipeline.from_pretrained(
    'stable-diffusion-v1-5/stable-diffusion-v1-5',
    torch_dtype=torch.float16,
    use_safetensors=True
).to('cuda')

Define 4 different `Generator`s and assign each `Generator` a seed. Then generate a batch of images and pick one to iterate on.

In [ ]:
generator = [torch.Generator('cuda').manual_seed(i) for i in range(4)]
prompt = "Labrador in the style of Vermeer"

images = pipeline(
    prompt,
    generator=generator,
    num_images_per_prompt=4,
).images[0]
make_image_grid(images, rows=2, cols=2)

Assume we choose the first image which corresponds to the `Generator` with seed `0`. We can add additional text to our prompt and reuse the same `Generator` with seed `0`. All the generated images should resemble the first image.

In [ ]:
prompt = [
    prompt + t
    for t in [", highly realistic", ", artsy", ", trending", ", colorful"]
]
generator = [torch.Generator('cuda').manual_seed(0) for _ in range(4)]

images = pipeline(
    prompt,
    generator=generator,
).images
make_image_grid(images, rows=2, cols=2)